In [11]:
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import mean_squared_error

In [7]:
dataset = pd.read_csv("/Users/micahokoko/Desktop/Micah-Portfolio/Fight_Prediction_Model/dataset/dataset.csv")
dataset = dataset[dataset['clean_outcome'].notna()]
dataset = dataset[dataset['clean_outcome'] != 'Unknown']
dataset['clean_outcome'] = dataset['clean_outcome'].map({'win': 1, 'loss': 0})


X = dataset[['opp_last6', 'opp_KnockedOut ratio', 'opp_winOther',  'opp_winKO', 'opptotalaccuracy', 'oppavgtotalagainst',
                    'opp_KO ratio', 'opp_loss', 'opp_lossOther', 'opp_win', 'opp_lossKO', 'oppavgtotallanded',
                    'Total punch accuracy', 'Avg Total punches landed', 'Avg Total punches landed against',
                    'Win KO', 'Loss KO', 'Win Other', 'KO ratio', 'Loss Other',
                    'KnockedOut ratio', 'Draw', 'last6']]
X = X.fillna(0)

Y = dataset['clean_outcome']
Y = Y.fillna(0)

# Get 60% of the dataset as the training set. Put the remaining 40% in temporary variables: x_ and y_.
x_train, x_, y_train, y_ = train_test_split(X, Y, test_size=0.40, random_state=1)

# Split the 40% subset above into two: one half for cross validation and the other for the test set
x_cv, x_test, y_cv, y_test = train_test_split(x_, y_, test_size=0.50, random_state=1)

# Delete temporary
del x_, y_

In [20]:
x_train = tf.convert_to_tensor(x_train, dtype=tf.float32)
y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)

x_test = tf.convert_to_tensor(x_test, dtype=tf.float32)
y_test = tf.convert_to_tensor(y_test, dtype=tf.float32)

x_cv = tf.convert_to_tensor(x_cv, dtype=tf.float32)
y_cv = tf.convert_to_tensor(y_cv, dtype=tf.float32)

In [21]:
# Create model
model = Sequential([
    Dense(units=20, activation='relu'),
    Dense(units=10, activation='relu'),
    Dense(units=1, activation='sigmoid')
])

# Compile model
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
              metrics=['accuracy'])

# Step 3: Train data to minimise cost function:
model.fit(x_train, y_train, epochs=100)

# Summary of the model
model.summary()

Epoch 1/100
70/70 [==============================] - 1s 2ms/step - loss: 0.8152 - accuracy: 0.7743
Epoch 2/100
70/70 [==============================] - 0s 3ms/step - loss: 0.4427 - accuracy: 0.8394
Epoch 3/100
70/70 [==============================] - 0s 2ms/step - loss: 0.4211 - accuracy: 0.8453
Epoch 4/100
70/70 [==============================] - 0s 3ms/step - loss: 0.3473 - accuracy: 0.8666
Epoch 5/100
70/70 [==============================] - 0s 3ms/step - loss: 0.3142 - accuracy: 0.8734
Epoch 6/100
70/70 [==============================] - 0s 3ms/step - loss: 0.3123 - accuracy: 0.8643
Epoch 7/100
70/70 [==============================] - 0s 3ms/step - loss: 0.3266 - accuracy: 0.8652
Epoch 8/100
70/70 [==============================] - 0s 2ms/step - loss: 0.2927 - accuracy: 0.8756
Epoch 9/100
70/70 [==============================] - 0s 2ms/step - loss: 0.3191 - accuracy: 0.8720
Epoch 10/100
70/70 [==============================] - 0s 2ms/step - loss: 0.2985 - accuracy: 0.8747
Epoch 11/

70/70 [==============================] - 0s 4ms/step - loss: 0.2274 - accuracy: 0.9005
Epoch 84/100
70/70 [==============================] - 0s 5ms/step - loss: 0.2355 - accuracy: 0.8933
Epoch 85/100
70/70 [==============================] - 0s 3ms/step - loss: 0.2464 - accuracy: 0.8942
Epoch 86/100
70/70 [==============================] - 0s 2ms/step - loss: 0.2442 - accuracy: 0.8978
Epoch 87/100
70/70 [==============================] - 0s 2ms/step - loss: 0.2445 - accuracy: 0.8996
Epoch 88/100
70/70 [==============================] - 0s 2ms/step - loss: 0.2405 - accuracy: 0.8951
Epoch 89/100
70/70 [==============================] - 0s 2ms/step - loss: 0.2351 - accuracy: 0.8969
Epoch 90/100
70/70 [==============================] - 0s 2ms/step - loss: 0.2344 - accuracy: 0.8978
Epoch 91/100
70/70 [==============================] - 0s 2ms/step - loss: 0.2314 - accuracy: 0.8937
Epoch 92/100
70/70 [==============================] - 0s 2ms/step - loss: 0.2309 - accuracy: 0.8996
Epoch 93/100


In [22]:
# Predict using the model
yhat_test = model.predict(x_test).flatten()
yhat_test = [1 if y > 0.5 else 0 for y in yhat_test]  # Convert probabilities to class labels

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, yhat_test)
precision = precision_score(y_test, yhat_test)
recall = recall_score(y_test, yhat_test)
f1 = f1_score(y_test, yhat_test)
roc_auc = roc_auc_score(y_test, yhat_test)

24/24 [==============================] - 0s 1ms/step


In [23]:
# Print the metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"AUC-ROC: {roc_auc:.4f}")

Accuracy: 0.8548
Precision: 0.8438
Recall: 0.9543
F1 Score: 0.8956
AUC-ROC: 0.8111


In [26]:
# Define the columns used as features and their corresponding 'opposition' versions
feature_columns = ['opp_last6', 'opp_KnockedOut ratio', 'opp_winOther',  'opp_winKO', 'opptotalaccuracy', 'oppavgtotalagainst',
                    'opp_KO ratio', 'opp_loss', 'opp_lossOther', 'opp_win', 'opp_lossKO', 'oppavgtotallanded',
                    'Total punch accuracy', 'Avg Total punches landed', 'Avg Total punches landed against',
                    'Win KO', 'Loss KO', 'Win Other', 'KO ratio', 'Loss Other',
                    'KnockedOut ratio', 'Draw', 'last6']



opposition_feature_mapping = {
    'opp_winKO': 'Win KO',
    'opp_winOther': 'Win Other',
    'opp_lossKO': 'Loss KO',
    'opp_lossOther': 'Loss Other',
    'opp_last6': 'last6',
    'opp_KO ratio': 'KO ratio',
    'opp_KnockedOut ratio': 'KnockedOut ratio',
    'opptotalaccuracy': 'Total punch accuracy',
    'oppavgtotalagainst': 'Avg Total punches landed against',
    'oppavgtotallanded': 'Avg Total punches landed'
}

# Function to create a new row for prediction
def create_prediction_row(fighter1, fighter2, dataset, feature_columns, opposition_feature_mapping):
    if fighter1 not in dataset['name'].values:
        raise ValueError(f"Fighter {fighter1} not found in dataset.")
    if fighter2 not in dataset['name'].values:
        raise ValueError(f"Fighter {fighter2} not found in dataset.")

    # Find the rows for fighter1 and fighter2
    row_fighter1 = dataset[dataset['name'] == fighter1].iloc[0]
    row_fighter2 = dataset[dataset['name'] == fighter2].iloc[0]

    # Create a new row with the structure of the dataset
    new_row = pd.DataFrame(columns=dataset.columns)
    new_row.loc[0, 'name'] = fighter1
    new_row.loc[0, 'opposition'] = fighter2

    # Populate the new row with features from both fighters
    for feature in feature_columns:
        if feature in opposition_feature_mapping:  # If it's an opposition feature
            mapped_feature = opposition_feature_mapping[feature]
            new_row.loc[0, feature] = row_fighter2[mapped_feature]
        else:  # If it's a regular feature
            new_row.loc[0, feature] = row_fighter1[feature]

    return new_row[feature_columns]

# Prediction function
def predict_outcome(fighter1, fighter2):
    # Create a new row for prediction
    prediction_row_f1_vs_f2 = create_prediction_row(fighter1, fighter2, dataset, feature_columns, opposition_feature_mapping)
    prediction_row_f2_vs_f1 = create_prediction_row(fighter2, fighter1, dataset, feature_columns, opposition_feature_mapping)

    prediction_features_f1_vs_f2 = prediction_row_f1_vs_f2.values.reshape(1, -1)
    prediction_features_f1_vs_f2 = tf.convert_to_tensor(prediction_features_f1_vs_f2, dtype=tf.float32)
    prediction_f1_vs_f2 = model.predict(prediction_features_f1_vs_f2)

    prediction_features_f2_vs_f1 = prediction_row_f2_vs_f1.values.reshape(1, -1)
    prediction_features_f2_vs_f1 = tf.convert_to_tensor(prediction_features_f2_vs_f1, dtype=tf.float32)
    prediction_f2_vs_f1 = model.predict(prediction_features_f2_vs_f1)

    # Determine the most likely winner
    if prediction_f1_vs_f2[0] > prediction_f2_vs_f1[0]:
        winner = fighter1
        looser = fighter2
        winning_chance = prediction_f1_vs_f2[0]
        looser_chance = prediction_f2_vs_f1[0]
    else:
        winner = fighter2
        looser = fighter1
        winning_chance = prediction_f2_vs_f1[0]
        looser_chance = prediction_f1_vs_f2[0]

    return winner, looser, winning_chance, looser_chance

In [25]:
# Example usage
fighter1_name = "Errol Spence Jr"
fighter2_name = "Terence Crawford"


try:
    winner, looser, winning_chance, looser_chance = predict_outcome(fighter1_name, fighter2_name)
    print(f"Predicted Winner: {winner} with a {winning_chance * 100}% chance to win")
    print(f"Predicted Looser: {looser} with a {looser_chance * 100}% chance to win")
except ValueError as e:
    print(e)

1/1 [==============================] - 0s 22ms/step
Predicted Winner: Terence Crawford with a [34.091137]% chance to win
Predicted Looser: Errol Spence Jr with a [24.047411]% chance to win
